Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Use home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory(sqllite)
4. Brave Search
5. Financial data(Polygon.io)

And a resource to read information about the trader's account, and their investment strategy.

Experiment and explore here, and then migrate to a python module <<traders.py>>(a single trader on our trading floor)

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


Create MCP params for our traders

In [3]:
# if you have a paid polygon account, use their mcp tools, otherwise use yours(created)
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

In [4]:
trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

Params for researcher

In [5]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

Create MCPServerStdio for each(trader and researcher)

In [6]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

Create a Researcher Agent to do market research

Note: In OpenAI Agents SDK agent could be used as a tool

In [7]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Reseacher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers
    )
    return researcher

In [8]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
        tool_name="Researcher",
        tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )



Try calling our research Agent for something

In [9]:
research_question = "What's the latest news on SoundHound.ai?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)

display(Markdown(result.final_output))

Here is the latest news summary on SoundHound.ai:

1. SoundHound AI recently released a study showing growing consumer interest in ordering food via in-vehicle voice assistants, highlighting a shift toward voice commerce. (Source: SoundHound press release)

2. The stock price of SoundHound AI (NASDAQ: SOUN) has seen a notable recent movement — it surged 69% month-to-date but then experienced a sharp pullback with a 10.26% drop on a recent trading day, marking the third consecutive day of declines. (Source: Yahoo Finance)

3. Despite strong customer acquisition and increased revenue guidance for 2025, SoundHound AI remains unprofitable and is experiencing widening losses. (Source: Yahoo Finance)

Overall, the company is making strides in voice commerce technology and market interest, but it faces profitability challenges at present, and its stock price has had significant volatility recently.

Would you like a deeper financial analysis or focus on recent developments in their products and market strategy?

Check the traces: https://platform.openai.com/traces

Create a Trader Agent

In [10]:
initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Gulnara").reset(initial_strategy)

display(Markdown(await read_accounts_resource("Gulnara")))
display(Markdown(await read_strategy_resource("Gulnara")))

{"name": "gulnara", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-21 09:32:39", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

In [11]:
agent_name = "Gulnara"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [12]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Gulnara and your account is under your name, Gulnara.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "gulnara", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-21 09:32:39", 10000.0], ["2025-08-21 09:32:42", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking

Run the Trader Agent

In [13]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4.1-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

I have executed an initial trade based on recent earnings information in the technology sector. Specifically, I bought 40 shares of MINISO Group Holding Limited (MNSO) at approximately $20.88 per share. This company showed a positive EPS surprise of +28.22%, which could indicate near-term upward momentum and volatility suitable for day trading.

I will continue to monitor the price movements and news for MNSO and other technology stocks with earnings updates to make further trades.

In [14]:
# check account
await read_accounts_resource(agent_name)

'{"name": "gulnara", "balance": 9164.7328, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"MNSO": 40}, "transactions": [{"symbol": "MNSO", "quantity": 40, "price": 20.88168, "timestamp": "2025-08-21 09:34:55", "rationale": "MINISO Group Holding Limited has a positive EPS surprise of +28.22%, indicating potential upward momentum and volatility suitable for aggressive day trading."}], "portfolio_value_time_series": [["2025-08-21 09:32:39", 10000.0], ["2025-08-21 09:32:42", 10000.0], ["2025-08-21 09:34:55", 9998.3328], ["2025-08-21 12:42:38", 9998.3328]], "total_portfolio_value": 9998.3328, "total_profit_loss": -1.6671999999998661}'

I created additional files to make it modular:
mcp_params.py` is where the MCP servers are specified. 

templates.py is where the instructions and messages are set up (i.e. the System prompts and User prompts)

traders.py, Trader class that brings all together

Lets call it:

In [15]:
from traders import Trader
trader = Trader("Gulnara")
await trader.run()

check balance

In [16]:
await read_accounts_resource("Gulnara")

# check traces also: https://platform.openai.com/traces

'{"name": "gulnara", "balance": 6569.8534, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"MNSO": 40, "PLTR": 10, "NVDA": 5, "SOHU": 10}, "transactions": [{"symbol": "MNSO", "quantity": 40, "price": 20.88168, "timestamp": "2025-08-21 09:34:55", "rationale": "MINISO Group Holding Limited has a positive EPS surprise of +28.22%, indicating potential upward momentum and volatility suitable for aggressive day trading."}, {"symbol": "PLTR", "quantity": 10, "price": 156.32201999999998, "timestamp": "2025-08-21 13:33:45", "rationale": "Palantir Technologies is showing strong upward momentum with a 461.61% return over the past year and is deeply embedded in the AI sector."}, {"symbol": "NVDA", "quantity": 5, "price": 175.7508, "timestamp": "2025-08-21 13:33:45", "rationale": "Nvidia continues to lead in AI technology, experiencing robust market growth and strong share price performance."}, {"symbol": "SOHU", "quantit

How many tools were used in total?

In [17]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
